#### Домашняя работа Урок 4. Рекоммендательные системы на основе контента. Поиск похожих товаров и пользователей

##### ВНИМАНИЕ!

Домашняя работе заканчивается после пункта
2.2 Создаем и обучаем модель BayesianPersonalizedRanking

В п. 2.4 Метрика precision_at_k здесь приведен расчет всех метрик по всем моделям данного курса.


Для того, чтобы эта модель заработала, потратил много времени. Она заработала!

#### Варианты для ранжирования (Краткое введение. Подробности - на следующем вебинаре)

**Вариант 1.**  
*As is* - ранжировать по output ALS

**Вариант 2.**
*Та же модель, другой лосс - BPR*

BPR - Bayesian Personalized Ranking loss
1. Случайная пара user - купленный им item_i = позитивная пара
2. Для того же юзера сэмплируем item_j (не купил или вес в user-item матрице ниже, у item_i из 1.) - негативная пара
3. Прогнозы модели $p_{ui}$, $p_{uj}$ --> $p_{ui} - p_{uj}$
4. loss = $\sigma(p_{ui} - p_{uj})$ - это и есть наша ошибка
5. Обновляем эмбеддинги

### 1. Базовое применение  BPR - Bayesian Personalized Ranking loss

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Для работы с матрицами
from scipy.sparse import csr_matrix


# Bayesian Personalized Ranking loss
from implicit.bpr import BayesianPersonalizedRanking

from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Матричная факторизация
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Метрики
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))

if module_path not in sys.path:
    sys.path.append(module_path)

import watermark
# from src.metrics import precision_at_k, recall_at_k

In [3]:
import sys
print("User Current Version:\n-",sys.version)

User Current Version:
- 3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]


In [4]:
implicit.__version__

'0.6.2'

In [5]:
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

%watermark -a 'ASV' -d -t -v -p implicit,numpy,scipy,pandas,matplotlib

Author: ASV

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 7.31.1

implicit  : 0.6.2
numpy     : 1.22.3
scipy     : 1.10.0
pandas    : 1.5.3
matplotlib: 3.6.3



### 1.1 Загрузим данные по проданным товарам

In [6]:
data = pd.read_csv('./data/retail_train.csv')

In [7]:
# data = pd.read_csv('../raw_data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

In [8]:
# подсчет количества различных элементов номеров недель
data['week_no'].nunique()

95

In [9]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('#Кол-во пользователей users: ', users)
print('#Кол-во товаров items: ', items)
print('#Взаимодействия interactions: ', interactions)

#Кол-во пользователей users:  2499
#Кол-во товаров items:  89051
#Взаимодействия interactions:  2396804


### 1.2 Train-test split

В рекомендательных системах корректнее использовать train-test split по времени, а не случайно.  
Возмем последние 3 недели в качестве теста

In [10]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [11]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [12]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


### 1.3 Загрузим данные информации о товарах

In [13]:
item_features = pd.read_csv('./data/product.csv')

In [14]:
# item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [15]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

*Результаты тестовых рекомендаций запишем в массив result*

In [16]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [17]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [18]:
popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [19]:
data_train.shape

(2278490, 12)

In [20]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


### 1.4 Подготовим матрицы и данные для ввода в модель

##### 1.4.1 Построим:
- сводную матрицу user_item_matrix, где строки 'user_id', столбцы 'item_id', в ячейках 0 или 1;
- переведем ее с формат saprse matrix;
- используем ее для построения всех моделей.

In [21]:
%%time

# Заведем фиктивный item_id (если юзер покупал товары НЕ из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id',
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1  
user_item_matrix = user_item_matrix.astype(float) 

# переведем в формат saprse matrix
# sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

Wall time: 1min 36s


item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*Посмотрим размерность матрицы user_item_matrix и % ее заполнения*

In [22]:
user_item_matrix.shape

(2499, 86865)

In [23]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

0.5804511409122447

#### Для обучения модли мы должны на вход подать разреженную матрицу sparse_user_item, где строки user а столбцы item. Здесь важен порядок. 

#### Переведем user_item_matrix в формат saprse matrix

In [24]:
%%time
# переведем user_item_matrix в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

Wall time: 5.48 s


##### 1.4.2 Построим:
- сводную матрицу user_item_matrix_test, где строки 'user_id', столбцы 'item_id', в ячейках 0 или 1;
- переведем ее в формат saprse matrix;
- используем ее для построения метрик всех моделей.

In [25]:
%%time

# Заведем фиктивный item_id (если юзер покупал товары НЕ из топ-5000, то он "купил" такой товар)
# data_test.loc[~data_test['item_id'].isin(top_5000), 'item_id'] = 999999
# data_test !

user_item_matrix_test = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix_test[user_item_matrix_test > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix_test = user_item_matrix_test.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
# sparse_user_item_test = csr_matrix(user_item_matrix).tocsr()

user_item_matrix_test.head(3)

Wall time: 12.3 s


item_id,29512,30356,32392,32439,34873,40405,42852,43094,44049,44522,...,17959192,17959243,17959460,17974316,17974317,17991689,17991691,18000012,18024155,18024556
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
user_item_matrix_test.shape

(2042, 24329)

In [27]:
# % заполнения user_item_matrix_test
user_item_matrix_test.sum().sum() / (user_item_matrix_test.shape[0] * user_item_matrix_test.shape[1]) * 100

0.21074352567072607

In [28]:
%%time

# переведем в формат saprse matrix
sparse_user_item_test = csr_matrix(user_item_matrix_test).tocsr()

Wall time: 693 ms


##### Подготовим дополнительные данные для формирования результатов и  работы с результатами рекомендаций

In [29]:
# userids = user_item_matrix.index.values
# itemids = user_item_matrix.columns.values

# matrix_userids = np.arange(len(userids))
# matrix_itemids = np.arange(len(itemids))

# id_to_itemid = dict(zip(matrix_itemids, itemids))
# id_to_userid = dict(zip(matrix_userids, userids))

# itemid_to_id = dict(zip(itemids, matrix_itemids))
# userid_to_id = dict(zip(userids, matrix_userids))

In [30]:
# Подготавливает вспомогательные словари

# перенумеруем пользователей и товары, 
# чтобы они задавались числами от 0 до номера количества товаров.

# userids array содержит индексы user_id, какие-то значения могут быть пропущены 
userids = user_item_matrix.index.values

# itemids array содержит индексы item_id, какие-то значения могут быть пропущены 
itemids = user_item_matrix.columns.values

# Формируем матрицы с последовательными числами, т.е. без пропусков
# номеров, перенумеруем для задания последовательных номеров userids, itemids
# числами от 0 до фактического количества userids, itemids

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))


# Формируем словари где ключ это перенумерованный порядковый номер userids, itemids
# значение ключа это фактическое значение из данных userids, itemids 
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# Другими словами, стовим фактическому номеру его порядковое значение
#  от 0 до n, подставляем в модель и снова переходим от порядковог номера к фактическому

## 2. Создаем и обучаем модели

### 2.1 Создаем и обучаем модель ItemItemRecommender. Базовый уровень.

In [31]:
%%time

model_ItemItem = ItemItemRecommender(K=25, num_threads=4)

Wall time: 0 ns


In [32]:
%%time

model_ItemItem.fit(sparse_user_item, show_progress=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

Wall time: 6.16 s


In [33]:
%%time
# введем порядковый номер user, начинается с 1
user_n = 1
id_n, scores_n = model_ItemItem.recommend(userid=userid_to_id[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5,
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

Wall time: 5.03 ms


In [34]:
id_n, scores_n

(array([34707, 23590, 36454,  7933, 25064]),
 array([75655., 63331., 55663., 54766., 54407.]))

In [35]:
res_n = [id_to_itemid[rec] for rec in id_n]
res_n

[1082185, 981760, 1098066, 840361, 995242]

In [36]:
%%time
precision_at_k(model_ItemItem, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 643 ms


0.0016492713666797951

### 2.2 Создаем и обучаем модель BayesianPersonalizedRanking

In [37]:
%%time

model_BPR = BayesianPersonalizedRanking(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                num_threads=4,
                                random_state=42)

model_BPR.fit(sparse_user_item, show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 5.49 s


##### Сделаем предсказание для одного user. BayesianPersonalizedRanking

In [38]:
%%time
# введем порядковый номер user, начинается с 0
# обращаем внимание на ввод userid=userids[user_n]
user_n = 2
id_BPR_n, score_BPR_n = model_BPR.recommend(userid=userids[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None)

Wall time: 155 ms


In [39]:
id_BPR_n, score_BPR_n

(array([34707, 23590, 36454,  6361,  7933]),
 array([2.507562 , 1.6095294, 1.3707888, 1.2939955, 1.2515785],
       dtype=float32))

In [40]:
res_BPR_n = [id_to_itemid[rec] for rec in id_BPR_n]
res_BPR_n

[1082185, 981760, 1098066, 826249, 840361]

##### Сделаем предсказание для всех user. Model BayesianPersonalizedRanking. Данный метод располагает результаты рекомендаций по убыванию, это видно по score, расположенных по убыванию

In [41]:
matrix_userids

array([   0,    1,    2, ..., 2496, 2497, 2498])

In [42]:
%%time

ids_BPR_all, scores_BPR_all = \
                model_BPR.recommend(userid=matrix_userids,
                N=5,
                user_items=sparse_user_item[matrix_userids])

Wall time: 989 ms


In [43]:
ids_BPR_all, scores_BPR_all

(array([[39619, 57637, 40692, 15537,  9570],
        [23590, 25064, 53360, 28895, 15011],
        [25620, 28895, 23590, 38628, 31558],
        ...,
        [39619, 23590, 53360, 25064, 36454],
        [40184, 28895, 11350, 38628, 38088],
        [ 9570, 16343, 12254, 15537, 10848]]),
 array([[1.7800694 , 1.5900397 , 1.5297248 , 1.506065  , 1.4316473 ],
        [1.4450669 , 1.0809221 , 1.0784523 , 0.9611357 , 0.920622  ],
        [2.2429254 , 2.2399087 , 2.2140656 , 2.204236  , 2.2041297 ],
        ...,
        [1.3557637 , 1.325986  , 1.0816007 , 1.0344565 , 0.96945   ],
        [2.2030299 , 2.0900145 , 2.007174  , 1.891937  , 1.819252  ],
        [1.0439229 , 0.8772954 , 0.8763663 , 0.85679436, 0.8447573 ]],
       dtype=float32))

In [44]:
# преобразуем данные для рекомендации user [2]
res_BPR_all = [id_to_itemid[rec] for rec in ids_BPR_all[2]]
res_BPR_all

[999971, 1029743, 981760, 1118533, 1053763]

!!! Ниже приведенный код не заработал. После введения вместо  userid=userid_to_id другую адресацию юзеров userid=matrix_userids Все заработало!

In [45]:
# Ниже приведенный код не заработал!
# %%time

# ids_BPR_all, scores_BPR_all = model_BPR.recommend(userid=userid_to_id,
#                         user_items=sparse_user_item,
#                         N=5, 
#                         filter_already_liked_items=False, 
#                         filter_items=None)

Каждая модель в implicit также имеет возможность отображать связанные элементы с помощью метода similar_items. Например, чтобы приобрести сопутствующие товары для itemid:

Вычислим список похожих элементов на itemid в колчестве N=5

In [46]:
itemid=19

itemids_BPR_top, scores_BPR_items_top= model_BPR.similar_items(itemid =itemid, N=5)

In [47]:
itemids_BPR_top, scores_BPR_items_top

(array([   19,   653, 81028, 30457, 65130]),
 array([1.0000001, 0.7750406, 0.7729453, 0.7672488, 0.7640645],
       dtype=float32))

Вычислим список похожих элементов на userid=19 в колчестве N=5. Первый элемент в списке будет сам userid=19. Список ранжирован.

In [48]:
userid=19

user_BPR_top, scores_BPR_user_top= model_BPR.similar_users(userid =userid, N=5)

In [49]:
user_BPR_top, scores_BPR_user_top

(array([  19, 1106,  583, 1684, 1242]),
 array([0.99999976, 0.9908    , 0.9888828 , 0.98867816, 0.9886654 ],
       dtype=float32))

#### precision_at_k

In [50]:
%%time
precision_at_k(model_BPR, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 1.01 s


0.0012800315084679006

# Выводы:
- Модель на основе BayesianPersonalizedRanking заработала;
- результаты метрик precision_at_k хуже чем у модели ItemItemRecommender, соответственно 0.001280 против 0.001649;
- применять модель на основе BayesianPersonalizedRanking надо, и в зависимости от требуемого быстродействия и точности она может быть предпрочтительнее других моделей.

# КОНЕЦ ДОМАШНЕЙ РАБОТЫ

### 2.2 Создаем и обучаем модель AlternatingLeastSquares

In [51]:
%%time

model_ALS = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
#                                     alpha=2.0,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                   random_state=42)


model_ALS.fit(sparse_user_item, show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 7.87 s


##### Сделаем предсказание для одного user. Model AlternatingLeastSquares

In [52]:
%%time
# введем порядковый номер user, начинается с 0
# обращаем внимание на ввод userid=userids[user_n]
user_n = 2
id_ALS_n, score_ALS_n = model_ALS.recommend(userid=userids[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

Wall time: 37.9 ms


In [53]:
id_ALS_n, score_ALS_n

(array([20372,  6361, 36454, 30505, 18934]),
 array([0.88953924, 0.8366004 , 0.8328201 , 0.83081454, 0.82302475],
       dtype=float32))

In [54]:
res_ALS_n = [id_to_itemid[rec] for rec in id_ALS_n]
res_ALS_n

[951590, 826249, 1098066, 1044078, 938700]

##### Сделаем предсказание для всех user. Model AlternatingLeastSquares. Данный метод располагает результаты рекомендаций по убыванию, это видно по score, расположенных по убыванию

In [55]:
%%time

ids_ALS_all, scores_ALS_all = model_ALS.recommend(userid=userid_to_id,
                        user_items=sparse_user_item,
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

Wall time: 869 ms


In [56]:
ids_ALS_all, scores_ALS_all

(array([[34707, 25064, 21566, 53360, 21930],
        [34707, 51478, 40184, 25064, 20372],
        [20372,  6361, 36454, 30505, 18934],
        ...,
        [25064, 31548, 28895, 33181, 34707],
        [36454, 12722,  6361, 34707, 18039],
        [53360, 55470, 34707, 10029, 32128]]),
 array([[0.7770564 , 0.77526283, 0.69948226, 0.6704397 , 0.6636717 ],
        [0.83727276, 0.7606707 , 0.7478005 , 0.7327474 , 0.7229275 ],
        [0.8895392 , 0.8366004 , 0.83281994, 0.83081424, 0.8230249 ],
        ...,
        [0.89464986, 0.81612056, 0.7802684 , 0.69296   , 0.68919694],
        [0.98912364, 0.9797734 , 0.96442646, 0.88717264, 0.8538709 ],
        [1.2803144 , 1.175401  , 1.1643715 , 1.054828  , 1.0529094 ]],
       dtype=float32))

In [57]:
# преобразуем данные для рекомендации user [2]
res_ALS_all = [id_to_itemid[rec] for rec in ids_ALS_all[2]]
res_ALS_all

[951590, 826249, 1098066, 1044078, 938700]

Каждая модель в implicit также имеет возможность отображать связанные элементы с помощью метода similar_items. Например, чтобы приобрести сопутствующие товары для itemid (the Beatles):

Вычислим список похожих элементов на itemid в колчестве N=5

In [58]:
itemid=19

itemids_ALS_top, scores_ALS_items_top= model_ALS.similar_items(itemid =itemid, N=5)

In [59]:
itemids_ALS_top, scores_ALS_items_top

(array([1376, 1153,  815,  710,  401]),
 array([1.0000001, 1.0000001, 1.0000001, 1.0000001, 1.0000001],
       dtype=float32))

In [60]:
%%time
precision_at_k(model_ALS, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 1.01 s


0.0013046474990153604

### 2.3 Создаем и обучаем модель AlternatingLeastSquares с улучшением tfidf и bm25 взвешиваним 

##### Улучшение модели:

Первым шагом в использовании этой модели будет преобразование необработанных показателей взаимодействиям из исходного набора данных в значения, которые можно использовать в качестве конфиденциальных данных. Придадим повторным воспроизведениям больше уверенности в модели, но сделать так, чтобы этот эффект уменьшался по мере увеличения количества повторных воспроизведений, чтобы уменьшить влияние одного суперпокупателя на модель. Аналогичным образом, уменьшим часть веса доверия от популярных товаров. Для этого мы будем использовать схему взвешивания tfidf и bm25, используемым классическим поиском информации:

### tfidf взвешивание

In [61]:
%%time

user_item_tfidf_weight = tfidf_weight(user_item_matrix)
user_item_tfidf_weight

Wall time: 5.64 s


<2499x86865 sparse matrix of type '<class 'numpy.float64'>'
	with 1260018 stored elements in COOrdinate format>

In [62]:
%%time
sparse_user_item_tfidf_weight = csr_matrix(user_item_tfidf_weight).tocsr()

Wall time: 8.97 ms


In [63]:
%%time

model_ALS_tfidf_weight = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
#                                     alpha=2.0,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                   random_state=42)


model_ALS_tfidf_weight.fit(sparse_user_item_tfidf_weight, show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 8.65 s


In [64]:
%%time

ids_ALS_all_tfidf_weight, scores_ALS_all_tfidf_weight = model_ALS_tfidf_weight.recommend(userid=userid_to_id,
                                                                                         user_items=sparse_user_item_tfidf_weight,
                                                                                         N=5,
                                                                                         filter_already_liked_items=False,
                                                                                         filter_items=None,recalculate_user=True)

Wall time: 1.6 s


In [65]:
ids_ALS_all_tfidf_weight, scores_ALS_all_tfidf_weight

(array([[33881, 36745, 21930, 34707, 28313],
        [27991, 34707, 33181, 16343, 39619],
        [30505, 20372, 63730, 20448, 36454],
        ...,
        [25064, 20333, 10029, 19166, 39283],
        [ 6361, 12722, 18039, 36454, 55470],
        [53360, 55470, 32128,  8311, 39528]]),
 array([[0.8729474 , 0.8117805 , 0.79779875, 0.79263747, 0.78578025],
        [0.72550213, 0.72091967, 0.71929574, 0.6902452 , 0.673291  ],
        [0.8253164 , 0.8089884 , 0.7548382 , 0.72153735, 0.72040224],
        ...,
        [0.886269  , 0.84516585, 0.8353957 , 0.79636604, 0.7814876 ],
        [1.0799088 , 1.0154603 , 0.93883216, 0.90032107, 0.8794851 ],
        [1.1919494 , 1.1568499 , 1.0915545 , 1.0399225 , 0.980589  ]],
       dtype=float32))

In [66]:
# преобразуем данные для рекомендации user [2]
res_ALS_all_tfidf_weight = [id_to_itemid[rec] for rec in ids_ALS_all_tfidf_weight[2]]
res_ALS_all_tfidf_weight

[1044078, 951590, 9337581, 952317, 1098066]

### BM25 взвешивание

In [67]:
%%time

user_item_bm25_weight = bm25_weight(user_item_matrix, K1=100, B=0.8)
user_item_bm25_weight

Wall time: 5.63 s


<2499x86865 sparse matrix of type '<class 'numpy.float64'>'
	with 1260018 stored elements in COOrdinate format>

In [68]:
%%time
sparse_user_item_bm25_weight = csr_matrix(user_item_bm25_weight).tocsr()

Wall time: 9.97 ms


In [69]:
%%time

model_ALS_bm25_weight = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
#                                     alpha=2.0,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                   random_state=42)


model_ALS_bm25_weight.fit(sparse_user_item_bm25_weight, show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 8.34 s


In [70]:
%%time

ids_ALS_all_bm25_weight, scores_ALS_all_bm25_weight = model_ALS_bm25_weight.recommend(userid=userid_to_id,
                                                                                         user_items=sparse_user_item_bm25_weight,
                                                                                         N=5,
                                                                                         filter_already_liked_items=False,
                                                                                         filter_items=None,
                                                                                      recalculate_user=True)

Wall time: 1.63 s


In [71]:
ids_ALS_all_bm25_weight, scores_ALS_all_bm25_weight

(array([[28313, 34707, 24179, 53360, 33881],
        [33181, 34707, 26401, 25064, 26161],
        [30505, 20372, 36454, 34498, 12722],
        ...,
        [36454, 39283,  7933, 40540, 10029],
        [ 6361, 18039, 36454, 15673, 30505],
        [53360, 53357, 16343,  8311, 58390]]),
 array([[0.98492986, 0.9161265 , 0.86325806, 0.8557593 , 0.8505762 ],
        [1.0521727 , 0.92197716, 0.8824277 , 0.8637313 , 0.8477476 ],
        [0.9049987 , 0.8905025 , 0.88356465, 0.8710272 , 0.8574017 ],
        ...,
        [0.9000511 , 0.89699495, 0.88428986, 0.81962353, 0.81057376],
        [1.0924671 , 0.98870146, 0.9654795 , 0.9480335 , 0.9400965 ],
        [1.1731818 , 0.81980973, 0.7251205 , 0.7134891 , 0.70432186]],
       dtype=float32))

In [72]:
# преобразуем данные для рекомендации user [2]
res_ALS_all_bm25_weight = [id_to_itemid[rec] for rec in ids_ALS_all_bm25_weight[2]]
res_ALS_all_bm25_weight

[1044078, 951590, 1098066, 1080414, 883404]

### 2.4 Метрика precision_at_k

In [79]:
%%time
precision_at_k(model_ItemItem, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 596 ms


0.0016492713666797951

In [74]:
%%time
precision_at_k(model_ALS, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 980 ms


0.0013046474990153604

In [75]:
%%time
precision_at_k(model_ALS_tfidf_weight, sparse_user_item_tfidf_weight, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 1 s


0.0012307995273729815

In [76]:
%%time
precision_at_k(model_ALS_bm25_weight, sparse_user_item_bm25_weight, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 1.02 s


0.001206183536825522

In [77]:
%%time
precision_at_k(model_BPR, sparse_user_item, sparse_user_item_test,
               K=25, show_progress=True, num_threads=4)

  0%|          | 0/2042 [00:00<?, ?it/s]

Wall time: 1.02 s


0.0012800315084679006

# 2.5  Вывод:
- модель model_ItemItem рекомендует лучше всех;
- веса предсказаний в AlternatingLeastSquares приведены к 0..1, что легче интерпретируется.

In [78]:
STOP

NameError: name 'STOP' is not defined

### Embeddings

Можно очень быстро посчитать предсказания, перемножив эти 2 матрицы

In [ ]:
model_ALS.item_factors.shape

In [ ]:
model_ALS.user_factors.shape

Можно очень быстро посчитать предсказания, перемножив эти 2 матрицы

In [ ]:
fast_recs = model_ALS.user_factors @ model_ALS.item_factors.T 
fast_recs.shape

In [ ]:
# fast_recs

In [ ]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [ ]:
# %%time
    
# result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
# result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

In [ ]:
# result.head(2)

### Embeddings

In [ ]:
# model.item_factors.shape

In [ ]:
# model.user_factors.shape

Можно очень быстро посчитать предсказания, перемножив эти 2 матрицы

In [ ]:
# fast_recs = model.user_factors @ model.item_factors.T 
# fast_recs.shape

## 3. Изобразим расположение товаров на 2D графике

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import cm
import pickle

from scipy.spatial.distance import cdist

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns


def reduce_dims(df, dims=2, method='pca'):
    
    assert method in ['pca', 'tsne'], 'Неверно указан метод'
    
    if method=='pca':
        pca = PCA(n_components=dims)
        components = pca.fit_transform(df)
    elif method == 'tsne':
        tsne = TSNE(n_components=dims, learning_rate=250, random_state=42)
        components = tsne.fit_transform(df)
    else:
        print('Error')
        
    colnames = ['component_' + str(i) for i in range(1, dims+1)]
    return pd.DataFrame(data = components, columns = colnames) 


def display_components_in_2D_space(components_df, labels='category', marker='D'):
    
    groups = components_df.groupby(labels)

    # Plot
    fig, ax = plt.subplots(figsize=(12,8))
    ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
    for name, group in groups:
        ax.plot(group.component_1, group.component_2, 
                marker='o', ms=6,
                linestyle='',
                alpha=0.7,
                label=name)
    ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5))

    plt.xlabel('component_1')
    plt.ylabel('component_2') 
    plt.show()

In [ ]:
model_ALS.item_factors.shape

In [ ]:
category = []

for idx in range(model_ALS.item_factors.shape[0]):

    try:
        cat = item_features.loc[item_features['item_id'] == id_to_itemid[idx], 'department'].values[0]
        category.append(cat)
    except:
        category.append('UNKNOWN')

In [ ]:
%%time
item_emb_tsne = reduce_dims(model_ALS.item_factors, dims=2, method='tsne') # 5001 х 100  ---> 5001 x 2
item_emb_tsne['category'] = category  # Добавляем категорию
item_emb_tsne = item_emb_tsne[item_emb_tsne['category'] != 'UNKNOWN']

display_components_in_2D_space(item_emb_tsne, labels='category')

Нарисуем все, кроме GROCERY

In [ ]:
display_components_in_2D_space(item_emb_tsne[item_emb_tsne['category'] != 'GROCERY'], labels='category')

Нарисуем несколько конкретных категорий

In [ ]:
interesting_cats = ['PASTRY', 'PRODUCE', 'DRUG GM', 'FLORAL']

display_components_in_2D_space(item_emb_tsne[item_emb_tsne['category'].isin(interesting_cats)], 
                                             labels='category')

На самом деле **результат средний**:
- Модель выучила похожесть только небольшой части товаров

In [ ]:
item_emb_tsne.head(2)

In [ ]:
# stop

recommend_all делает перемножение, но еще и сортирует и выбирает топ-N

## 4. Важно искать оптимальные параметры

- regularization, iterations
- factors
- Вес (элемент в матрице user-item)

-----

# 4.1  Вывод:
- модель model_ItemItem рекомендует лучше всех;
- веса предсказаний в AlternatingLeastSquares приведены к 0..1, что легче интерпретируется.

# Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его сущесвтенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере iFood. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влязает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [ ]:
data_train.head(2)

In [ ]:
data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))
data_train['price'].max()

In [ ]:
# < 1$
data_train['price'].quantile(0.20)

In [ ]:
# > 100$
data_train['price'].quantile(0.99995)

In [ ]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товарыs
    
    # ...
    
def postfilter_items(user_id, recommednations):
    pass

Все эти функции отправим затем в *src.utils*

----

In [ ]:
# !pip install watermark

In [ ]:
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%watermark -a 'ASV' -d -t -v -p numpy,pandas,matplotlib,implicit